In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import re
import shutil
import time
from datetime import datetime
from datetime import timedelta
pd.set_option('display.max_columns',None)

In [49]:
'''更改默认路径'''
os.getcwd()
os.chdir('C:\\Users\\Administrator\\desktop\\python data  lib\\单笔')

In [50]:
'''导入指定数据文件'''
hqdata=pd.read_csv('单笔非变更.csv',sep='\t')
sbdata=pd.read_csv('单笔变更.csv',sep='\t')

In [51]:
'''修改字段格式'''
hqdata['客户编号']=hqdata['客户编号'].astype('object')
hqdata['批复日期']=pd.to_datetime(hqdata['批复日期'])
sbdata['客户编号']=sbdata['客户编号'].astype('object')
sbdata['批复日期']=pd.to_datetime(sbdata['批复日期'])

In [52]:
'''确保无空值'''
sbdata.loc[sbdata['批复日期'].isnull(),:]
hqdata.loc[hqdata['批复日期'].isnull(),:]
hqdata.dtypes

客户编号                         object
审批机构级别                       object
申报性质                         object
业务编号                         object
批复日期                 datetime64[ns]
批复授信/明细支用金额（合人民币）           float64
审批结论                         object
客户名称                         object
dtype: object

In [53]:
'''组内排序'''
hqdata['sort_id']=hqdata['批复日期'].groupby(hqdata['客户编号']).rank(method='dense',ascending=True)
hqdata=hqdata.sort_values(by=['客户编号','sort_id'])
hqdata=hqdata.reset_index(drop=True)
hqdata.loc[hqdata['客户编号']=='224890000000106323',:]

,客户编号,审批机构级别,申报性质,业务编号,批复日期,批复授信/明细支用金额（合人民币）,审批结论,客户名称,sort_id


In [54]:
'''组内排序'''
sbdata['sort_id']=sbdata['批复日期'].groupby(sbdata['客户编号']).rank(method='dense',ascending=True)
sbdata=sbdata.sort_values(by=['客户编号','sort_id'])
sbdata=sbdata.reset_index(drop=True)
sbdata.loc[sbdata['客户编号']=='224890000000106323',:]

,客户编号,审批机构级别,申报性质,业务编号,批复日期,批复授信/明细支用金额（合人民币）,审批结论,客户名称,sort_id


In [55]:
'''对总行数据进行转置变换'''
# hqdata.loc[hqdata['客户编号']=='464801000002085800',:]
max_id=hqdata['sort_id'].max()
for id in range(1,int(max_id)+1):
    if id==1:
        hqdata_tranpose=hqdata.loc[hqdata['sort_id']==id,['客户编号','业务编号','批复日期']]
    else:
        hqdata_tranpose1=hqdata.loc[hqdata['sort_id']==id,['客户编号','业务编号','批复日期']]
        hqdata_tranpose=pd.merge(hqdata_tranpose,hqdata_tranpose1,on='客户编号',how='left',suffixes=('_'+str(id-1),'_'+str(id)))
        
hqdata_tranpose.loc[~hqdata_tranpose['业务编号_'+str(int(max_id))].isnull(),:]
hqdata_tranpose.loc[hqdata_tranpose['客户编号']=='224890000000106323',:]
max_id

34.0

In [56]:
'''总分行数据连接 选择内连接模式'''
sbdata_join=pd.merge(sbdata,hqdata_tranpose,on='客户编号',how='inner',suffixes=('_变更','_'+str(max_id)))
sbdata_join.loc[sbdata_join['客户编号']=='224890000000106323',:]

,客户编号,审批机构级别,申报性质,业务编号,批复日期,批复授信/明细支用金额（合人民币）,审批结论,客户名称,sort_id,业务编号_1,批复日期_1,业务编号_2,批复日期_2,业务编号_3,批复日期_3,业务编号_4,批复日期_4,业务编号_5,批复日期_5,业务编号_6,批复日期_6,业务编号_7,批复日期_7,业务编号_8,批复日期_8,业务编号_9,批复日期_9,业务编号_10,批复日期_10,业务编号_11,批复日期_11,业务编号_12,批复日期_12,业务编号_13,批复日期_13,业务编号_14,批复日期_14,业务编号_15,批复日期_15,业务编号_16,批复日期_16,业务编号_17,批复日期_17,业务编号_18,批复日期_18,业务编号_19,批复日期_19,业务编号_20,批复日期_20,业务编号_21,批复日期_21,业务编号_22,批复日期_22,业务编号_23,批复日期_23,业务编号_24,批复日期_24,业务编号_25,批复日期_25,业务编号_26,批复日期_26,业务编号_27,批复日期_27,业务编号_28,批复日期_28,业务编号_29,批复日期_29,业务编号_30,批复日期_30,业务编号_31,批复日期_31,业务编号_32,批复日期_32,业务编号_33,批复日期_33,业务编号_34,批复日期_34


In [57]:
'''建立字段 所对应总行审批业务编号'''
sbdata_join['所对应总行审批业务编号']=np.nan
sbdata_join['所对应总行审批业务批复时间']=np.nan
sbdata_join.loc[sbdata_join['客户编号']=='224890000000106323',:]

,客户编号,审批机构级别,申报性质,业务编号,批复日期,批复授信/明细支用金额（合人民币）,审批结论,客户名称,sort_id,业务编号_1,批复日期_1,业务编号_2,批复日期_2,业务编号_3,批复日期_3,业务编号_4,批复日期_4,业务编号_5,批复日期_5,业务编号_6,批复日期_6,业务编号_7,批复日期_7,业务编号_8,批复日期_8,业务编号_9,批复日期_9,业务编号_10,批复日期_10,业务编号_11,批复日期_11,业务编号_12,批复日期_12,业务编号_13,批复日期_13,业务编号_14,批复日期_14,业务编号_15,批复日期_15,业务编号_16,批复日期_16,业务编号_17,批复日期_17,业务编号_18,批复日期_18,业务编号_19,批复日期_19,业务编号_20,批复日期_20,业务编号_21,批复日期_21,业务编号_22,批复日期_22,业务编号_23,批复日期_23,业务编号_24,批复日期_24,业务编号_25,批复日期_25,业务编号_26,批复日期_26,业务编号_27,批复日期_27,业务编号_28,批复日期_28,业务编号_29,批复日期_29,业务编号_30,批复日期_30,业务编号_31,批复日期_31,业务编号_32,批复日期_32,业务编号_33,批复日期_33,业务编号_34,批复日期_34,所对应总行审批业务编号,所对应总行审批业务批复时间


In [58]:
'''建立函数 判断对应所属总行业务编号'''
def cut_box_target(box_no=1):
    cut_no=max(range(1,box_no+1))
    for i in range(int(sbdata_join.index.max())+1):
            if  cut_no>1 and cut_no<int(max_id) and pd.isna(sbdata_join.loc[i,'业务编号_'+str(cut_no+1)]) :
                for j in range(1,cut_no):
                    if sbdata_join.loc[i,'批复日期']>=sbdata_join.loc[i,'批复日期_'+str(j)]   \
                    and sbdata_join.loc[i,'批复日期']<sbdata_join.loc[i,'批复日期_'+str(j+1)]:
                        sbdata_join.loc[i,'所对应总行审批业务编号']=sbdata_join.loc[i,'业务编号_'+str(j)]
                        sbdata_join.loc[i,'所对应总行审批业务批复时间']=sbdata_join.loc[i,'批复日期_'+str(j)]
                if  sbdata_join.loc[i,'批复日期']>=sbdata_join.loc[i,'批复日期_'+str(cut_no)]:
                    sbdata_join.loc[i,'所对应总行审批业务编号']=sbdata_join.loc[i,'业务编号_'+str(cut_no)]
                    sbdata_join.loc[i,'所对应总行审批业务批复时间']=sbdata_join.loc[i,'批复日期_'+str(cut_no)]
            elif cut_no==1 and pd.isna(sbdata_join.loc[i,'业务编号_'+str(cut_no+1)]) : 
                 if  sbdata_join.loc[i,'批复日期']>=sbdata_join.loc[i,'批复日期_'+str(cut_no)]:
                        sbdata_join.loc[i,'所对应总行审批业务编号']=sbdata_join.loc[i,'业务编号_'+str(cut_no)]
                        sbdata_join.loc[i,'所对应总行审批业务批复时间']=sbdata_join.loc[i,'批复日期_'+str(cut_no)]
            elif cut_no==int(max_id)  : 
                for j in range(1,cut_no):
                    if sbdata_join.loc[i,'批复日期']>=sbdata_join.loc[i,'批复日期_'+str(j)]   \
                    and sbdata_join.loc[i,'批复日期']<sbdata_join.loc[i,'批复日期_'+str(j+1)]:
                        sbdata_join.loc[i,'所对应总行审批业务编号']=sbdata_join.loc[i,'业务编号_'+str(j)]
                        sbdata_join.loc[i,'所对应总行审批业务批复时间']=sbdata_join.loc[i,'批复日期_'+str(j)]
                if  sbdata_join.loc[i,'批复日期']>=sbdata_join.loc[i,'批复日期_'+str(cut_no)]:
                    sbdata_join.loc[i,'所对应总行审批业务编号']=sbdata_join.loc[i,'业务编号_'+str(cut_no)]
                    sbdata_join.loc[i,'所对应总行审批业务批复时间']=sbdata_join.loc[i,'批复日期_'+str(cut_no)]
    return sbdata_join

for i in range(1,int(max_id)+1):
    cut_box_target(i)
    
sbdata_join.loc[sbdata_join['客户编号']=='224890000000106323',:]

,客户编号,审批机构级别,申报性质,业务编号,批复日期,批复授信/明细支用金额（合人民币）,审批结论,客户名称,sort_id,业务编号_1,批复日期_1,业务编号_2,批复日期_2,业务编号_3,批复日期_3,业务编号_4,批复日期_4,业务编号_5,批复日期_5,业务编号_6,批复日期_6,业务编号_7,批复日期_7,业务编号_8,批复日期_8,业务编号_9,批复日期_9,业务编号_10,批复日期_10,业务编号_11,批复日期_11,业务编号_12,批复日期_12,业务编号_13,批复日期_13,业务编号_14,批复日期_14,业务编号_15,批复日期_15,业务编号_16,批复日期_16,业务编号_17,批复日期_17,业务编号_18,批复日期_18,业务编号_19,批复日期_19,业务编号_20,批复日期_20,业务编号_21,批复日期_21,业务编号_22,批复日期_22,业务编号_23,批复日期_23,业务编号_24,批复日期_24,业务编号_25,批复日期_25,业务编号_26,批复日期_26,业务编号_27,批复日期_27,业务编号_28,批复日期_28,业务编号_29,批复日期_29,业务编号_30,批复日期_30,业务编号_31,批复日期_31,业务编号_32,批复日期_32,业务编号_33,批复日期_33,业务编号_34,批复日期_34,所对应总行审批业务编号,所对应总行审批业务批复时间


In [59]:
'''生成对应excel文件'''
df_data=sbdata_join.loc[~sbdata_join['所对应总行审批业务编号'].isnull(),['客户编号','客户名称', \
                                                      '业务编号',  '批复日期', '所对应总行审批业务编号','所对应总行审批业务批复时间']]
df_data_result=sbdata_join.groupby('所对应总行审批业务编号')['业务编号'].count()
df_data=pd.merge(df_data,df_data_result,on='所对应总行审批业务编号',how='left',suffixes=('_变更业务','_发送变更统计'))
writer=pd.ExcelWriter( '统计结果.xlsx')
df_data.to_excel(writer,'Sheet1')
df_data_result.to_excel(writer,'Sheet2')
writer.save()